# Configurations

In [2]:
from IPython.display import display, HTML
import numpy as np
import pandas as pd

Test_file     = './Dataset/test_66516Ee.csv'

Test_dtype = {'id':np.str, 'perc_premium_paid_by_cash_credit':np.float32, 
'age_in_days':np.float32, 
'Income':np.float32,
'Count_3-6_months_late':np.float32, 
'Count_6-12_months_late':np.float32,
'Count_more_than_12_months_late':np.float32, 
'application_underwriting_score':np.float32,
'no_of_premiums_paid':np.float32, 
'sourcing_channel':np.str, 
'residence_area_type':np.str,
'premium':np.int}

## Loading dataset

In [53]:
# Loading dataset
tst_origin = pd.read_csv(Test_file, dtype=Test_dtype, na_values='')

display(tst_origin.head())

tst_data = tst_origin[['id', 'premium']].copy()

#Check the shape of each dataset
print(tst_data.shape)

,id,perc_premium_paid_by_cash_credit,age_in_days,Income,Count_3-6_months_late,Count_6-12_months_late,Count_more_than_12_months_late,application_underwriting_score,no_of_premiums_paid,sourcing_channel,residence_area_type,premium
0,649,0.001,27384.0,51150.0,0.0,0.0,0.0,99.889999,7.0,A,Rural,3300
1,81136,0.124,23735.0,285140.0,0.0,0.0,0.0,98.930000,19.0,A,Urban,11700
2,70762,1.000,17170.0,186030.0,0.0,0.0,0.0,NaN,2.0,B,Urban,11700
3,53935,0.198,16068.0,123540.0,0.0,0.0,0.0,99.000000,11.0,B,Rural,5400
4,15476,0.041,10591.0,200020.0,1.0,0.0,0.0,99.169998,14.0,A,Rural,9600


(34224, 2)


### Arbitrarily put renewal probabilities for each policy

In [163]:
pd_renewal = pd.DataFrame(np.random.rand(len(tst_data),1), columns=['renewal'])
pd_incentives = pd.DataFrame(np.zeros((len(tst_data), 1)), columns=['incentives'], dtype=np.float32)
pd_improve = pd.DataFrame(np.zeros((len(tst_data), 1)), columns=['improvement'], dtype=np.float32)
tst_data2 = pd.concat([tst_data, pd_renewal, pd_improve, pd_incentives], axis=1)


display(tst_data2.head())
tst_data2.shape

,id,premium,renewal,improvement,incentives
0,649,3300,0.856611,0.0,0.0
1,81136,11700,0.786612,0.0,0.0
2,70762,11700,0.802665,0.0,0.0
3,53935,5400,0.313039,0.0,0.0
4,15476,9600,0.060244,0.0,0.0


(34224, 5)

### Build equations

In [155]:
import math

def inc_to_eff(inc):
    return 10*(1-np.exp(-1.0*inc/400.0))

def eff_to_imp(eff):
    return 20*(1-np.exp(-1.0*eff/5.0))/100

# def derivative(inc, delta):
#     y1 = eff_to_imp(inc_to_eff(inc))
#     y2 = eff_to_imp(inc_to_eff(inc+delta))
#     return (y2-y1)/delta

def rev_policy(renewal, premium, inc):
    #print('--', renewal, eff_to_imp(inc_to_eff(inc)), premium, inc )
    return (renewal + renewal*eff_to_imp(inc_to_eff(inc))) * premium - inc

def derivatives(renewal, premium, inc, delta=1):
    y1 = rev_policy(renewal, premium, inc)
    y2 = rev_policy(renewal, premium, inc+delta)
    return (y2-y1)/delta

def total_net_rev(dataset):
    return dataset['']


In [147]:
print(inc_to_eff(1000))
print(eff_to_imp(10))
print(eff_to_imp(inc_to_eff(100)))
print(derivatives(0.5, 10000, 100, 1))
print(derivatives(0.5, 1000, 100, 1))
print(derivatives(0.8, 10000, 100, 1))
print(derivatives(0.5, 10000, 10, 1))


9.179150013761012
0.17293294335267745
0.0715012813499204
3.9877902676344092
-0.5012209732366273
3.9877902676344092
8.249050858639748


### Ok, Let's find optimal solution!!!!

In [165]:
import time

#unit_inc = 100
learning_rate = 0.02
delta = 1
MAX_ITER = 100000
t1 = time.time()

itr = 0
prev_total_net_rev = 0
total_net_rev = 0

while True:
    if itr > 2  and total_net_rev-prev_total_net_rev < 1:
        print('Total net revenue is below 1!')
        break
    if itr > MAX_ITER:
        print('Maximum iteration reached.')
        break
    result = derivatives(tst_data2['renewal'], tst_data2['premium'], tst_data2['incentives'], delta)
    #print(result.head())
    cand_idx, deriv = result.idxmax(), result.max()
    #data.loc[data.bidder == 'parakeet2004', 'bidderrate'] = 100
    tst_data2.loc[cand_idx, 'incentives'] = tst_data2.loc[cand_idx, 'incentives'] + learning_rate * deriv
    tst_data2.loc[cand_idx, 'improvement'] = eff_to_imp(inc_to_eff(tst_data2.loc[cand_idx, 'incentives']))
    
    prev_total_net_rev = total_net_rev
    total_net_rev = (tst_data2['renewal'] + tst_data2['renewal']*tst_data2['improvement']) * tst_data2['premium'] - tst_data2['incentives']
    total_net_rev = np.sum(total_net_rev)
    
    if itr%100 == 0:
        print('%d\t%d\t%13.3f\t%13.3f\t%13.3f' %(itr, cand_idx, deriv, total_net_rev, total_net_rev-prev_total_net_rev))
    
    itr += 1

print(round(time.time() - t1), 'seconds.')
    
#np.exp(1/tst_data2['premium'])

0	12472	       49.426	184634628.086	184634628.086
100	19911	       48.753	184639458.751	       47.542
200	25784	       48.173	184644153.845	       46.419
300	17139	       47.552	184648726.282	       45.231
400	19432	       46.948	184653190.731	       44.092
500	24555	       46.469	184657554.837	       43.200
600	11678	       45.954	184661825.301	       42.249
700	25784	       45.532	184666011.168	       41.477
800	945	       45.082	184670114.974	       40.663
900	14997	       44.685	184674143.655	       39.950
1000	33672	       44.291	184678102.688	       39.250
1100	12640	       43.923	184681995.144	       38.602
1200	18299	       43.575	184685824.672	       37.994
1300	2864	       43.240	184689594.533	       37.413
1400	10321	       42.924	184693307.332	       36.868
1500	17393	       42.592	184696964.580	       36.300
1600	4211	       42.282	184700568.604	       35.776
1700	4633	       41.987	184704122.157	       35.279
1800	12071	       41.711	184707627.427	       34.816
1900	1784	

15500	2596	       29.021	185017305.516	       16.869
15600	25784	       28.984	185018990.579	       16.826
15700	22957	       28.949	185020671.558	       16.785
15800	8389	       28.907	185022347.635	       16.738
15900	25784	       28.870	185024019.271	       16.693
16000	25478	       28.835	185025686.720	       16.653
16100	2596	       28.793	185027349.555	       16.605
16200	19432	       28.755	185029008.025	       16.561
16300	25996	       28.720	185030662.325	       16.523
16400	17577	       28.680	185032312.358	       16.477
16500	2892	       28.645	185033958.029	       16.437
16600	4607	       28.609	185035599.595	       16.396
16700	3647	       28.571	185037237.026	       16.351
16800	25842	       28.537	185038870.169	       16.312
16900	19330	       28.502	185040499.296	       16.271
17000	32008	       28.463	185042124.287	       16.229
17100	2650	       28.432	185043745.162	       16.192
17200	2604	       28.396	185045362.265	       16.152
17300	5226	       28.359	185046975.3

30800	29114	       25.031	185237302.928	       12.555
30900	14932	       25.013	185238557.440	       12.536
31000	31514	       24.995	185239810.072	       12.519
31100	32103	       24.976	185241061.044	       12.500
31200	18913	       24.960	185242310.125	       12.484
31300	14851	       24.942	185243557.364	       12.466
31400	643	       24.923	185244802.918	       12.445
31500	33749	       24.907	185246046.733	       12.428
31600	6957	       24.889	185247288.872	       12.411
31700	6037	       24.871	185248529.163	       12.395
31800	8448	       24.854	185249767.713	       12.377
31900	30275	       24.837	185251004.611	       12.362
32000	31105	       24.821	185252239.798	       12.343
32100	16199	       24.803	185253473.287	       12.324
32200	7992	       24.785	185254705.067	       12.310
32300	14227	       24.769	185255935.131	       12.291
32400	21815	       24.754	185257163.591	       12.279
32500	32565	       24.737	185258390.421	       12.259
32600	25386	       24.718	18525961

46100	7837	       22.898	185412248.623	       10.505
46200	27950	       22.887	185413298.873	       10.498
46300	3592	       22.877	185414348.167	       10.487
46400	22714	       22.867	185415396.543	       10.479
46500	31926	       22.855	185416443.817	       10.468
46600	27913	       22.845	185417490.101	       10.458
46700	8465	       22.833	185418535.408	       10.448
46800	14368	       22.822	185419579.655	       10.438
46900	16478	       22.811	185420622.915	       10.428
47000	28375	       22.802	185421665.222	       10.418
47100	25487	       22.790	185422706.661	       10.407
47200	28029	       22.779	185423747.030	       10.397
47300	11496	       22.769	185424786.368	       10.389
47400	21770	       22.759	185425824.800	       10.380
47500	25378	       22.746	185426862.210	       10.368
47600	9923	       22.737	185427898.589	       10.360
47700	6037	       22.727	185428934.066	       10.351
47800	10888	       22.717	185429968.668	       10.342
47900	15010	       22.704	1854310

61400	11938	       21.455	185562614.402	        9.227
61500	20004	       21.447	185563536.620	        9.219
61600	9948	       21.440	185564458.164	        9.214
61700	14977	       21.431	185565379.022	        9.205
61800	273	       21.422	185566299.186	        9.197
61900	30763	       21.414	185567218.563	        9.191
62000	28669	       21.406	185568137.251	        9.184
62100	24253	       21.398	185569055.307	        9.178
62200	5768	       21.390	185569972.612	        9.170
62300	24504	       21.382	185570889.275	        9.163
62400	25533	       21.375	185571805.261	        9.156
62500	26707	       21.366	185572720.601	        9.150
62600	29514	       21.358	185573635.200	        9.142
62700	6398	       21.350	185574549.127	        9.136
62800	10093	       21.342	185575462.345	        9.130
62900	28180	       21.334	185576374.881	        9.123
63000	31410	       21.326	185577286.740	        9.114
63100	19739	       21.318	185578197.934	        9.109
63200	25120	       21.310	1855791

76700	4963	       20.358	185696369.047	        8.307
76800	9948	       20.353	185697199.564	        8.304
76900	11801	       20.346	185698029.568	        8.299
77000	27659	       20.341	185698859.098	        8.294
77100	20129	       20.333	185699688.040	        8.288
77200	14227	       20.327	185700516.447	        8.280
77300	12347	       20.320	185701344.337	        8.277
77400	3590	       20.314	185702171.711	        8.269
77500	7933	       20.308	185702998.613	        8.265
77600	7158	       20.302	185703824.993	        8.263
77700	1777	       20.296	185704650.895	        8.257
77800	968	       20.291	185705476.341	        8.253
77900	18898	       20.285	185706301.320	        8.248
78000	1495	       20.278	185707125.754	        8.243
78100	27763	       20.271	185707949.682	        8.235
78200	19384	       20.265	185708773.080	        8.229
78300	388	       20.259	185709595.935	        8.226
78400	12249	       20.252	185710418.313	        8.221
78500	29005	       20.247	185711240.209

92000	11113	       19.488	185817951.897	        7.613
92100	32169	       19.483	185818712.952	        7.610
92200	25069	       19.478	185819473.629	        7.604
92300	1787	       19.473	185820233.897	        7.601
92400	25493	       19.467	185820993.739	        7.597
92500	23996	       19.462	185821753.195	        7.591
92600	26047	       19.457	185822512.263	        7.589
92700	27950	       19.452	185823270.879	        7.585
92800	32368	       19.446	185824029.123	        7.580
92900	4767	       19.441	185824786.922	        7.576
93000	26342	       19.436	185825544.298	        7.573
93100	32565	       19.431	185826301.321	        7.567
93200	7507	       19.427	185827057.993	        7.566
93300	6918	       19.421	185827814.281	        7.561
93400	30103	       19.416	185828570.125	        7.555
93500	26224	       19.411	185829325.563	        7.554
93600	16026	       19.405	185830080.623	        7.549
93700	9683	       19.400	185830835.274	        7.545
93800	22933	       19.395	1858315

In [166]:
display(tst_data2)

,id,premium,renewal,improvement,incentives
0,649,3300,0.856611,0.000000,0.000000
1,81136,11700,0.786612,0.000000,0.000000
2,70762,11700,0.802665,0.000000,0.000000
3,53935,5400,0.313039,0.000000,0.000000
4,15476,9600,0.060244,0.000000,0.000000
5,64797,11700,0.267464,0.000000,0.000000
6,67412,3300,0.044806,0.000000,0.000000
7,44241,5400,0.697533,0.000000,0.000000
8,5069,13800,0.197291,0.000000,0.000000
9,16615,28500,0.968664,0.037208,43.445545


In [169]:
max(tst_data2['incentives'])

165.60205078125